In [3]:
%pip install tree_sitter groq networkx jinja2 python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 635.4/635.4 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 10.3 MB/s eta 0:00:00


In [ ]:
import os
import re
import subprocess
import tempfile
import time
import shutil
import zipfile
from pathlib import Path
from typing import List, Dict, Optional
import json
from groq import Groq

BACKEND_EXTENSIONS = {'.py', '.go', '.rs', '.swift'}

def clone_repo(repo_url: str, target_dir: str) -> bool:
    try:
        subprocess.run(
            ['git', 'clone', '--depth', '1', repo_url, target_dir],
            check=True,
            capture_output=True,
            text=True
        )
        return True
    except subprocess.CalledProcessError:
        return False

def find_backend_files(directory: str) -> List[str]:
    backend_files = []
    for root, _, files in os.walk(directory):
        for file in files:
            if Path(file).suffix in BACKEND_EXTENSIONS:
                backend_files.append(os.path.join(root, file))
    return backend_files

def extract_python_functions(content: str) -> List[str]:
    functions = []
    lines = content.split('\n')
    i = 0
    while i < len(lines):
        line = lines[i]
        stripped = line.lstrip()
        if stripped.startswith('def ') or stripped.startswith('async def '):
            indent = len(line) - len(stripped)
            func_lines = [line]
            i += 1
            while i < len(lines):
                current_line = lines[i]
                if current_line.strip() == '':
                    func_lines.append(current_line)
                    i += 1
                    continue
                current_indent = len(current_line) - len(current_line.lstrip())
                if current_indent <= indent and current_line.strip():
                    break
                func_lines.append(current_line)
                i += 1
            functions.append('\n'.join(func_lines))
            continue
        i += 1
    return functions

def extract_go_functions(content: str) -> List[str]:
    functions = []
    pattern = r'func\s+(?:\([^)]*\)\s+)?(\w+)\s*\([^)]*\)(?:\s*\([^)]*\)|\s+[^{]+)?\s*\{'
    for match in re.finditer(pattern, content):
        start = match.start()
        brace_count = 0
        func_start = start
        started = False
        for i in range(start, len(content)):
            if content[i] == '{':
                brace_count += 1
                started = True
            elif content[i] == '}':
                brace_count -= 1
                if brace_count == 0 and started:
                    functions.append(content[func_start:i+1])
                    break
    return functions

def extract_rust_functions(content: str) -> List[str]:
    functions = []
    pattern = r'(?:pub\s+)?(?:async\s+)?fn\s+\w+(?:<[^>]+>)?\s*\([^)]*\)(?:\s*->\s*[^{]+)?\s*\{'
    for match in re.finditer(pattern, content):
        start = match.start()
        brace_count = 0
        func_start = start
        started = False
        for i in range(start, len(content)):
            if content[i] == '{':
                brace_count += 1
                started = True
            elif content[i] == '}':
                brace_count -= 1
                if brace_count == 0 and started:
                    functions.append(content[func_start:i+1])
                    break
    return functions

def extract_swift_functions(content: str) -> List[str]:
    functions = []
    pattern = r'(?:public\s+|private\s+|internal\s+|fileprivate\s+)?(?:static\s+)?func\s+\w+(?:<[^>]+>)?\s*\([^)]*\)(?:\s*(?:throws|rethrows))?\s*(?:->\s*[^{]+)?\s*\{'
    for match in re.finditer(pattern, content):
        start = match.start()
        brace_count = 0
        func_start = start
        started = False
        for i in range(start, len(content)):
            if content[i] == '{':
                brace_count += 1
                started = True
            elif content[i] == '}':
                brace_count -= 1
                if brace_count == 0 and started:
                    functions.append(content[func_start:i+1])
                    break
    return functions

def extract_functions_from_file(filepath: str) -> List[str]:
    with open(filepath, 'r', encoding='utf-8', errors='ignore') as f:
        content = f.read()

    ext = Path(filepath).suffix
    if ext == '.py':
        return extract_python_functions(content)
    elif ext == '.go':
        return extract_go_functions(content)
    elif ext == '.rs':
        return extract_rust_functions(content)
    elif ext == '.swift':
        return extract_swift_functions(content)
    return []

def clean_llm_response(response: str) -> str:
    """Clean LLM response to extract pure JSON."""
    if not response or not response.strip():
        return ""

    response = re.sub(r'^```(?:json)?\s*\n', '', response, flags=re.MULTILINE)
    response = re.sub(r'\n```\s*$', '', response, flags=re.MULTILINE)
    response = response.strip()

    first_brace = response.find('{')
    last_brace = response.rfind('}')

    if first_brace != -1 and last_brace != -1 and last_brace > first_brace:
        response = response[first_brace:last_brace+1]

    return response

def generate_api_doc_with_llm(function_code: str, language: str, groq_api_key: str, max_retries: int = 3) -> Optional[str]:
    """Generate API documentation with retry logic."""
    client = Groq(api_key=groq_api_key)

    prompt = f"""Analyze this {language} function and generate comprehensive professional API documentation.

You MUST respond with ONLY valid JSON. Do not include any markdown code blocks, explanations, or text outside the JSON.

Generate JSON with these exact fields:

{{
  "function_name": "name of the function",
  "description": "Detailed 2-4 sentence description",
  "method": "HTTP method or FUNCTION CALL",
  "endpoint": "API endpoint or N/A",
  "examples": [
    {{
      "title": "Example title",
      "code": "Full code example",
      "description": "What this demonstrates"
    }}
  ],
  "parameters": [
    {{
      "name": "param_name",
      "type": "String",
      "required": "?",
      "constraints": "min_length=5",
      "description": "Detailed parameter description"
    }}
  ],
  "returns": {{
    "description": "What is returned",
    "type": "return_type",
    "fields": [],
    "sample_response": "{{}}"
  }},
  "errors": [
    {{
      "code": "ERROR_CODE",
      "description": "What causes this",
      "resolution": "How to fix"
    }}
  ],
  "notes": "Important notes and best practices"
}}

Function code:
```{language}
{function_code[:1000]}
```

Remember: Return ONLY the JSON object, nothing else."""

    for attempt in range(max_retries):
        try:
            chat_completion = client.chat.completions.create(
                messages=[{"role": "user", "content": prompt}],
                model="llama-3.1-8b-instant",
                temperature=0.3,
                max_tokens=4000,
            )

            response = chat_completion.choices[0].message.content

            if not response or not response.strip():
                print(f"    Attempt {attempt + 1}: Empty response, retrying...")
                time.sleep(2 ** attempt)
                continue

            return response

        except Exception as e:
            print(f"    Attempt {attempt + 1}: API error - {e}")
            if attempt < max_retries - 1:
                time.sleep(2 ** attempt)
            else:
                return None

    return None

def create_github_readme(repo_url: str, docs: List[Dict]) -> str:
    """Create a proper GitHub-style README."""
    repo_name = repo_url.rstrip('/').split('/')[-1].replace('-', ' ').title()

    # Calculate statistics
    by_language = {}
    for doc in docs:
        lang = doc.get('language', 'unknown')
        by_language[lang] = by_language.get(lang, 0) + 1

    readme = f"""<div align="center">

# {repo_name} API Documentation

**Auto-generated API documentation for the {repo_name} project**

[![Documentation](https://img.shields.io/badge/docs-latest-blue.svg)](./docs/)
[![Functions](https://img.shields.io/badge/functions-{len(docs)}-green.svg)]()
[![Languages](https://img.shields.io/badge/languages-{len(by_language)}-orange.svg)]()

[View Documentation](#documentation) • [Quick Start](#quick-start) • [API Reference](#api-reference)

</div>

---

## Overview

This repository contains comprehensive API documentation for **{repo_name}**, automatically generated from source code analysis. The documentation includes detailed function descriptions, parameters, return values, examples, and best practices.

**Generated on:** {time.strftime('%B %d, %Y at %H:%M:%S')}
**Source Repository:** [{repo_url}]({repo_url})

## Features

- **{len(docs)} Functions Documented** - Complete coverage of all backend functions
- **Detailed Analysis** - Parameters, return types, error handling, and examples
- **Multi-Language Support** - Documentation for {', '.join(sorted(by_language.keys())).upper()}
- **Structured Format** - Easy-to-navigate Markdown and JSON formats
- **Docusaurus Ready** - Deploy as a static documentation website

## Documentation Structure

```
api_docs/
├── README.md                      # This file
├── docs/
│   ├── intro.md                   # Introduction
│   ├── getting-started.md         # Getting started guide
│   └── api/                       # API reference by language
│       ├── python.md
│       ├── go.md
│       ├── rust.md
│       └── swift.md
├── docusaurus/                    # Docusaurus website source
└── api_documentation.json         # Raw JSON data
```

## 📊 Statistics

| Language | Functions | Percentage |
|----------|-----------|------------|
"""

    total = len(docs)
    for lang in sorted(by_language.keys()):
        count = by_language[lang]
        percentage = (count / total * 100) if total > 0 else 0
        readme += f"| {lang.upper()} | {count} | {percentage:.1f}% |\n"

    readme += f"""
**Total Functions:** {len(docs)}

## Quick Start

### View Documentation

1. **Browse by Language:**
"""

    for lang in sorted(by_language.keys()):
        readme += f"   - [{lang.upper()} Functions](./docs/api/{lang}.md) ({by_language[lang]} functions)\n"

    readme += """
2. **Search in JSON:**
   ```bash
   # Search for specific function
   cat api_documentation.json | jq '.[] | select(.function_name | contains("your_function"))'
   ```

3. **Deploy Documentation Website:**
   ```bash
   cd docusaurus
   npm install
   npm start
   # Visit http://localhost:3000
   ```

## API Reference

### By Language
"""

    for lang in sorted(by_language.keys()):
        readme += f"\n#### [{lang.upper()}](./docs/api/{lang}.md)\n\n"
        lang_docs = [d for d in docs if d.get('language') == lang]
        for i, doc in enumerate(lang_docs[:5], 1):
            func_name = doc.get('function_name', 'Unknown')
            desc = doc.get('description', 'No description')
            if len(desc) > 100:
                desc = desc[:97] + '...'
            readme += f"- **`{func_name}`** - {desc}\n"

        if len(lang_docs) > 5:
            readme += f"\n*... and {len(lang_docs) - 5} more functions*\n"

    readme += """

## Usage

### As JSON Data

```python
import json

# Load documentation
with open('api_documentation.json', 'r') as f:
    docs = json.load(f)

# Find specific function
function = next((d for d in docs if d['function_name'] == 'your_function'), None)
print(function['description'])
```

### As Static Website

```bash
# Build Docusaurus site
cd docusaurus
npm run build

# Serve locally
npm run serve

# Deploy (example with GitHub Pages)
npm run deploy
```

## Export Formats

This documentation is available in multiple formats:

- **Markdown** - Human-readable `.md` files in `docs/` directory
- **JSON** - Machine-readable `api_documentation.json`
- **Docusaurus** - Static website in `docusaurus/` directory
- **ZIP Archive** - `api_docs.zip` containing all documentation

## Contributing

Found an issue or want to improve the documentation?

1. Check the source code in the [original repository]({repo_url})
2. Submit issues or pull requests
3. Re-generate documentation after code changes

## License

This documentation follows the same license as the source repository.

## Acknowledgments

- Documentation generated using AI-powered analysis
- Built with [Docusaurus](https://docusaurus.io/)
- Powered by [Groq](https://groq.com/)

---

<div align="center">

**[Back to Top](#-{repo_name.lower().replace(' ', '-')}-api-documentation)**

Made with automated documentation tools

</div>
"""

    return readme

def create_docusaurus_structure(docs: List[Dict], repo_url: str, output_dir: str):
    """Create Docusaurus documentation structure."""
    docusaurus_dir = os.path.join(output_dir, 'docusaurus')
    os.makedirs(docusaurus_dir, exist_ok=True)

    repo_name = repo_url.rstrip('/').split('/')[-1]

    # Create package.json
    package_json = {
        "name": f"{repo_name}-docs",
        "version": "1.0.0",
        "private": True,
        "scripts": {
            "docusaurus": "docusaurus",
            "start": "docusaurus start",
            "build": "docusaurus build",
            "swizzle": "docusaurus swizzle",
            "deploy": "docusaurus deploy",
            "clear": "docusaurus clear",
            "serve": "docusaurus serve",
            "write-translations": "docusaurus write-translations",
            "write-heading-ids": "docusaurus write-heading-ids"
        },
        "dependencies": {
            "@docusaurus/core": "^3.0.0",
            "@docusaurus/preset-classic": "^3.0.0",
            "@mdx-js/react": "^3.0.0",
            "clsx": "^2.0.0",
            "prism-react-renderer": "^2.1.0",
            "react": "^18.2.0",
            "react-dom": "^18.2.0"
        },
        "devDependencies": {
            "@docusaurus/module-type-aliases": "^3.0.0",
            "@docusaurus/types": "^3.0.0"
        },
        "browserslist": {
            "production": [">0.5%", "not dead", "not op_mini all"],
            "development": ["last 1 chrome version", "last 1 firefox version", "last 1 safari version"]
        },
        "engines": {
            "node": ">=18.0"
        }
    }

    with open(os.path.join(docusaurus_dir, 'package.json'), 'w') as f:
        json.dump(package_json, f, indent=2)

    # Create docusaurus.config.js
    config_js = f"""// @ts-check
// Note: type annotations allow type checking and IDEs autocompletion

const lightCodeTheme = require('prism-react-renderer').themes.github;
const darkCodeTheme = require('prism-react-renderer').themes.dracula;

/** @type {{import('@docusaurus/types').Config}} */
const config = {{
  title: '{repo_name} API Documentation',
  tagline: 'Comprehensive API Documentation',
  favicon: 'img/favicon.ico',
  url: 'https://your-docusaurus-site.com',
  baseUrl: '/',
  organizationName: 'your-org',
  projectName: '{repo_name}-docs',
  onBrokenLinks: 'warn',
  onBrokenMarkdownLinks: 'warn',

  i18n: {{
    defaultLocale: 'en',
    locales: ['en'],
  }},

  presets: [
    [
      'classic',
      /** @type {{import('@docusaurus/preset-classic').Options}} */
      ({{
        docs: {{
          sidebarPath: require.resolve('./sidebars.js'),
          editUrl: '{repo_url}/tree/main/',
        }},
        theme: {{
          customCss: require.resolve('./src/css/custom.css'),
        }},
      }}),
    ],
  ],

  themeConfig:
    /** @type {{import('@docusaurus/preset-classic').ThemeConfig}} */
    ({{
      navbar: {{
        title: '{repo_name} Docs',
        items: [
          {{
            type: 'docSidebar',
            sidebarId: 'tutorialSidebar',
            position: 'left',
            label: 'API Reference',
          }},
          {{
            href: '{repo_url}',
            label: 'GitHub',
            position: 'right',
          }},
        ],
      }},
      footer: {{
        style: 'dark',
        copyright: `Copyright © ${{new Date().getFullYear()}} {repo_name}. Built with Docusaurus.`,
      }},
      prism: {{
        theme: lightCodeTheme,
        darkTheme: darkCodeTheme,
        additionalLanguages: ['python', 'go', 'rust', 'swift'],
      }},
    }}),
}};

module.exports = config;
"""

    with open(os.path.join(docusaurus_dir, 'docusaurus.config.js'), 'w') as f:
        f.write(config_js)

    # Create sidebars.js
    by_language = {}
    for doc in docs:
        lang = doc.get('language', 'unknown')
        if lang not in by_language:
            by_language[lang] = []
        by_language[lang].append(doc)

    sidebar_items = []
    for lang in sorted(by_language.keys()):
        sidebar_items.append(f"'api/{lang}'")

    sidebars_js = f"""// @ts-check

/** @type {{import('@docusaurus/plugin-content-docs').SidebarsConfig}} */
const sidebars = {{
  tutorialSidebar: [
    'intro',
    'getting-started',
    {{
      type: 'category',
      label: 'API Reference',
      items: [{', '.join(sidebar_items)}],
    }},
  ],
}};

module.exports = sidebars;
"""

    with open(os.path.join(docusaurus_dir, 'sidebars.js'), 'w') as f:
        f.write(sidebars_js)

    # Create docs directory
    docs_dir = os.path.join(docusaurus_dir, 'docs')
    os.makedirs(docs_dir, exist_ok=True)

    # Create intro.md
    intro_md = f"""---
sidebar_position: 1
---

# Introduction

Welcome to the **{repo_name}** API documentation! This comprehensive guide provides detailed information about all functions, their parameters, return values, and usage examples.

## What's Inside

This documentation covers **{len(docs)} functions** across multiple programming languages:

"""
    for lang in sorted(by_language.keys()):
        intro_md += f"- **{lang.upper()}**: {len(by_language[lang])} functions\n"

    intro_md += f"""

## Navigation

Use the sidebar to browse documentation by programming language, or use the search feature to find specific functions.

## Source Repository

All code is available at: [{repo_url}]({repo_url})

## Getting Help

If you find issues or have questions:

1. Check the function examples in each API reference
2. Review error handling sections
3. Consult the notes and best practices
4. Visit the source repository for more context
"""

    with open(os.path.join(docs_dir, 'intro.md'), 'w') as f:
        f.write(intro_md)

    # Create getting-started.md
    getting_started = """---
sidebar_position: 2
---

# Getting Started

## Quick Start Guide

This guide will help you understand and use the API documentation effectively.

### Understanding the Documentation

Each function is documented with:

- **Description**: What the function does
- **Parameters**: Input parameters with types and constraints
- **Returns**: Return value and structure
- **Examples**: Code examples showing usage
- **Errors**: Possible error conditions
- **Notes**: Best practices and important information

### Code Examples

All code examples are syntax-highlighted and ready to use. Simply copy and paste them into your project.

### Search Functionality

Use the search bar at the top of the page to quickly find functions by name or keyword.

## Next Steps

Browse the API reference in the sidebar to explore available functions.
"""

    with open(os.path.join(docs_dir, 'getting-started.md'), 'w') as f:
        f.write(getting_started)

    # Create API directory
    api_dir = os.path.join(docs_dir, 'api')
    os.makedirs(api_dir, exist_ok=True)

    # Create language-specific docs
    for lang, lang_docs in by_language.items():
        create_docusaurus_api_page(lang, lang_docs, api_dir)

    # Create src directory structure
    src_dir = os.path.join(docusaurus_dir, 'src')
    os.makedirs(os.path.join(src_dir, 'css'), exist_ok=True)

    custom_css = """/**
 * Custom CSS for API Documentation
 */

:root {
  --ifm-color-primary: #2e8555;
  --ifm-color-primary-dark: #29784c;
  --ifm-color-primary-darker: #277148;
  --ifm-color-primary-darkest: #205d3b;
  --ifm-color-primary-light: #33925d;
  --ifm-color-primary-lighter: #359962;
  --ifm-color-primary-lightest: #3cad6e;
  --ifm-code-font-size: 95%;
  --docusaurus-highlighted-code-line-bg: rgba(0, 0, 0, 0.1);
}

[data-theme='dark'] {
  --docusaurus-highlighted-code-line-bg: rgba(0, 0, 0, 0.3);
}
"""

    with open(os.path.join(src_dir, 'css', 'custom.css'), 'w') as f:
        f.write(custom_css)

    # Create static directory
    static_dir = os.path.join(docusaurus_dir, 'static')
    os.makedirs(os.path.join(static_dir, 'img'), exist_ok=True)

    # Create README for Docusaurus
    docusaurus_readme = f"""# {repo_name} Documentation Website

This is the Docusaurus documentation website for {repo_name}.

## Installation

```bash
npm install
```

## Local Development

```bash
npm start
```

This command starts a local development server and opens a browser window. Most changes are reflected live without restarting the server.

## Build

```bash
npm run build
```

This command generates static content into the `build` directory.

## Deployment

```bash
npm run serve
```

Serve the built website locally to test before deployment.

For GitHub Pages deployment:
```bash
GIT_USER=<Your GitHub username> npm run deploy
```
"""

    with open(os.path.join(docusaurus_dir, 'README.md'), 'w') as f:
        f.write(docusaurus_readme)

def create_docusaurus_api_page(language: str, docs: List[Dict], api_dir: str):
    """Create API reference page for a specific language in Docusaurus format."""
    md = f"""---
sidebar_position: {['python', 'go', 'rust', 'swift'].index(language) + 3 if language in ['python', 'go', 'rust', 'swift'] else 10}
---

# {language.upper()} API Reference

This page documents all {len(docs)} {language.upper()} functions in the codebase.

---

"""

    for i, doc in enumerate(docs, 1):
        md += f"\n## {doc.get('function_name', 'Unknown')}\n\n"

        if 'description' in doc:
            md += f"{doc['description']}\n\n"

        if 'method' in doc and doc['method'] != 'FUNCTION CALL':
            md += f"**Method:** `{doc['method']}`  \n"

        if 'endpoint' in doc and doc['endpoint'] != 'N/A':
            md += f"**Endpoint:** `{doc['endpoint']}`\n\n"

        if 'parameters' in doc and doc['parameters']:
            md += "### Parameters\n\n"
            for param in doc['parameters']:
                if isinstance(param, dict):
                    name = param.get('name', '')
                    ptype = param.get('type', '')
                    desc = param.get('description', '')
                    md += f"- **`{name}`** *({ptype})* - {desc}\n"
            md += "\n"

        if 'returns' in doc:
            md += "### Returns\n\n"
            ret = doc['returns']
            if isinstance(ret, dict):
                md += f"{ret.get('description', '')}\n\n"
                if 'sample_response' in ret:
                    sample = ret['sample_response']
                    if isinstance(sample, str):
                        md += f"```\n{sample}\n```\n\n"
                    else:
                        md += f"```json\n{json.dumps(sample, indent=2)}\n```\n\n"

        if 'examples' in doc and doc['examples']:
            md += "### Examples\n\n"
            for ex in doc['examples']:
                if isinstance(ex, dict):
                    if 'title' in ex:
                        md += f"**{ex['title']}**\n\n"
                    if 'code' in ex:
                        md += f"```{language}\n{ex['code']}\n```\n\n"
                    if 'description' in ex:
                        md += f"{ex['description']}\n\n"

        if 'errors' in doc and doc['errors']:
            md += "### Errors\n\n"
            for error in doc['errors']:
                if isinstance(error, dict):
                    md += f"- **{error.get('code', '')}**: {error.get('description', '')}\n"
            md += "\n"

        if 'notes' in doc:
            md += ":::info\n"
            md += f"{doc['notes']}\n"
            md += ":::\n\n"

        md += f"**Source:** `{doc.get('source_file', 'Unknown')}`\n\n"
        md += "---\n\n"

    with open(os.path.join(api_dir, f'{language}.md'), 'w') as f:
        f.write(md)

def create_zip_archive(output_dir: str):
    """Create a ZIP archive of all documentation."""
    zip_path = f'{output_dir}.zip'

    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(output_dir):
            for file in files:
                file_path = os.path.join(root, file)
                arcname = os.path.relpath(file_path, output_dir)
                zipf.write(file_path, arcname)

    return zip_path

def process_repository(repo_url: str, groq_api_key: str) -> List[Dict]:
    with tempfile.TemporaryDirectory() as tmpdir:
        print(f"Cloning repository: {repo_url}")
        if not clone_repo(repo_url, tmpdir):
            print("Failed to clone repository")
            return []

        print("Finding backend files...")
        backend_files = find_backend_files(tmpdir)
        print(f"Found {len(backend_files)} backend files")

        if not backend_files:
            print("No backend files found!")
            return []

        all_docs = []
        for filepath in backend_files:
            print(f"\nProcessing: {filepath}")
            try:
                functions = extract_functions_from_file(filepath)
                print(f"  Found {len(functions)} functions")

                if not functions:
                    continue

                language = Path(filepath).suffix[1:]

                for idx, func in enumerate(functions, 1):
                    try:
                        print(f"  Generating docs for function {idx}/{len(functions)}...")
                        doc_response = generate_api_doc_with_llm(func, language, groq_api_key)

                        if not doc_response:
                            print(f"    Failed to generate documentation (no response)")
                            continue

                        cleaned_response = clean_llm_response(doc_response)

                        if not cleaned_response:
                            print(f"    Failed to extract JSON from response")
                            continue

                        try:
                            doc_json = json.loads(cleaned_response)
                            doc_json['source_file'] = filepath.replace(tmpdir, '')
                            doc_json['language'] = language
                            doc_json['function_code'] = func
                            all_docs.append(doc_json)
                            print(f"    ✓ Successfully generated docs")
                        except json.JSONDecodeError as je:
                            print(f"    JSON parse error: {je}")
                            print(f"    Response preview: {cleaned_response[:200]}...")
                            all_docs.append({
                                'source_file': filepath.replace(tmpdir, ''),
                                'language': language,
                                'raw_documentation': cleaned_response,
                                'function_code': func,
                                'function_name': func.split('(')[0].split()[-1] if '(' in func else 'Unknown'
                            })
                    except Exception as e:
                        print(f"    Error processing function: {e}")

                    time.sleep(0.5)

            except Exception as e:
                print(f"  Error reading file: {e}")

        return all_docs

def generate_all_documentation(docs: List[Dict], repo_url: str, output_dir: str = 'api_docs'):
    """Generate all documentation formats."""
    os.makedirs(output_dir, exist_ok=True)

    print("\nGenerating documentation...")

    # 1. Create GitHub README
    print("  - Creating GitHub README...")
    readme_content = create_github_readme(repo_url, docs)
    with open(os.path.join(output_dir, 'README.md'), 'w', encoding='utf-8') as f:
        f.write(readme_content)

    # 2. Create Markdown docs directory
    print("  - Creating Markdown documentation...")
    docs_dir = os.path.join(output_dir, 'docs')
    os.makedirs(docs_dir, exist_ok=True)

    # Group by language
    by_language = {}
    for doc in docs:
        lang = doc.get('language', 'unknown')
        if lang not in by_language:
            by_language[lang] = []
        by_language[lang].append(doc)

    # Create intro
    with open(os.path.join(docs_dir, 'intro.md'), 'w', encoding='utf-8') as f:
        f.write(f"# Introduction\n\nComplete API documentation for {len(docs)} functions.\n")

    # Create API directory
    api_dir = os.path.join(docs_dir, 'api')
    os.makedirs(api_dir, exist_ok=True)

    # Create language-specific docs
    for lang, lang_docs in by_language.items():
        create_docusaurus_api_page(lang, lang_docs, api_dir)

    # 3. Create Docusaurus structure
    print("  - Creating Docusaurus website...")
    create_docusaurus_structure(docs, repo_url, output_dir)

    # 4. Create ZIP archive
    print("  - Creating ZIP archive...")
    zip_path = create_zip_archive(output_dir)

    print(f"\nDocumentation generated successfully!")
    print(f"\nOutput:")
    print(f"  Main directory: {output_dir}/")
    print(f"  README: {output_dir}/README.md")
    print(f"  Docs: {output_dir}/docs/")
    print(f"  Docusaurus: {output_dir}/docusaurus/")
    print(f"  ZIP archive: {zip_path}")
    print(f"\nTo start Docusaurus:")
    print(f"  cd {output_dir}/docusaurus")
    print(f"  npm install")
    print(f"  npm start")

if __name__ == "__main__":
    REPO_URL = "https://github.com/anwitac246/healthcare-ai-agent"
    GROQ_API_KEY = "API-KEY"

    print("Starting API documentation generation...\n")

    docs = process_repository(REPO_URL, GROQ_API_KEY)

    if docs:
        # Save JSON
        with open('api_documentation.json', 'w', encoding='utf-8') as f:
            json.dump(docs, f, indent=2)
        print(f"\nJSON saved to api_documentation.json")

        # Generate all documentation
        generate_all_documentation(docs, REPO_URL)
    else:
        print("\nNo documentation generated. Check errors above.")

Starting API documentation generation...

Cloning repository: https://github.com/anwitac246/healthcare-ai-agent
Finding backend files...
Found 5 backend files

Processing: /tmp/tmp54dymp7e/document.py
  Found 8 functions
  Generating docs for function 1/8...
    ✓ Successfully generated docs
  Generating docs for function 2/8...
    ✓ Successfully generated docs
  Generating docs for function 3/8...
    ✓ Successfully generated docs
  Generating docs for function 4/8...
    JSON parse error: Expecting ',' delimiter: line 63 column 3 (char 2105)
    Response preview: {
  "function_name": "rule_based_extraction",
  "description": "Performs rule-based extraction on a given text to identify various entities such as symptoms, conditions, medications, tests, dates, num...
  Generating docs for function 5/8...
    ✓ Successfully generated docs
  Generating docs for function 6/8...
    JSON parse error: Expecting ',' delimiter: line 43 column 3 (char 1931)
    Response preview: {
  "function_n